In [ ]:
#default_exp rescore.feature_extractor

In [2]:
import pandas as pd
import numpy as np

from alphadeep.model.ccs import AlphaCCSModel
from alphadeep.model.rt import AlphaRTModel
from alphadeep.model.ms2 import pDeepModel, calc_ms2_similarity
from alphadeep.mass_spec.match import PepSpecMatch
from alphabase.peptide.fragment import get_charged_frag_types

def rt_uniform_sampling(psm_df, n_train=1000, return_test_df=False):
    x = np.arange(0, 11)/10*psm_df.rt_norm.max()
    sub_n = n_train//(len(x)-1)
    df_list = []
    for i in range(len(x)-1):
        _df = psm_df[
            (psm_df.rt_norm>=x[i])&(psm_df.rt_norm<x[i+1])
        ]
        if len(_df) == 0: pass
        elif len(_df)//2 < sub_n:
            df_list.append(_df.sample(len(_df)//2))
        else:
            df_list.append(_df.sample(sub_n))
    train_df = pd.concat(df_list)
    if return_test_df:
        test_df = psm_df.drop(train_df.index)
        return train_df, test_df
    else:
        return train_df

class ScoreFeatureExtractor(object):
    def __init__(self,
        ms2_model:pDeepModel,
        rt_model:AlphaRTModel,
        ccs_model: AlphaCCSModel,
        ppm=True, tol=20,
        use_GPU = True,
    ):
        self.ms2_model = ms2_model
        self.rt_model = rt_model
        self.ccs_model = ccs_model
        self.ppm = ppm
        self.tol = tol
        self.GPU = use_GPU

        self.score_feature_list = [
            'COS', 'SA', 'SPC',
            'rt_delta', 'ccs_delta',
        ]

    def _fine_tune_models(self, 
        psm_df:pd.DataFrame, 
        matched_inten_df:pd.DataFrame
    ):
        pass

    def extract_features(self,
        psm_df: pd.DataFrame,
        ms2_file_dict, #raw_name: ms2_file_path or ms_reader object
        ms2_type:str = 'alphapept', #or 'mgf', or 'thermo'
        frag_types_to_match:list = get_charged_frag_types(['b','y'], 2),
        ms2_ppm=True, ms2_tol=20,
    ):
        self.match = PepSpecMatch(psm_df, 
            charged_frag_types=frag_types_to_match
        )

        self.match.match_ms2_centroid(
            ms2_file_dict=ms2_file_dict,
            ms2_type=ms2_type, 
            ppm=ms2_ppm, tol=ms2_tol,
        )

        self.psm_df = self.match.psm_df
        self.matched_mz_err_df = self.match.matched_mz_err_df
        self.matched_intensity_df = self.match.matched_intensity_df

        self.psm_df = self.rt_model.predict(
            self.psm_df
        )

        self.psm_df = self.ccs_model.predict(
            self.psm_df
        )

        self.predict_intensity_df = self.ms2_model.predict(
            self.psm_df, reference_frag_df=self.matched_intensity_df
        )
        pred_frag_types = []
        for frag_type in frag_types_to_match:
            if frag_type in self.ms2_model.charged_frag_types:
                pred_frag_types.append(frag_type)
        self.predict_intensity_df = self.predict_intensity_df[pred_frag_types]

        self.psm_df, ms2_metrics_df = calc_ms2_similarity(
            self.psm_df, self.predict_intensity_df, 
            self.match.matched_inten_df,
            charged_frag_types=pred_frag_types,
            metrics=['COS','SA','SPC'],
            GPU=self.GPU,
        )

        self.psm_df[
            'rt_delta'
        ] = (
            self.psm_df.rt_norm-self.psm_df.rt_pred
        ).abs()

        if self.psm_df.ccs.isna().any():
            self.psm_df['ccs_delta'] = 0
        else:
            self.psm_df[
                'ccs_delta'
            ] = (
                self.psm_df.ccs-self.psm_df.ccs_pred
            ).abs()

            